# <center>Домашнее задание № 10 (демо)
## <center> Прогнозирование задержек вылетов

Ваша задача – побить единственный бенчмарк в [соревновании](https://www.kaggle.com/c/flight-delays-2017) на Kaggle Inclass. Подробных инструкций не будет, будет только тезисно описано, как получен этот бенчмарк. Конечно, с помощью Xgboost. Надеюсь, на данном этапе курса вам достаточно бросить полтора взгляда на данные, чтоб понять, что это тот тип задачи, в которой затащит Xgboost. Но проверьте еще Catboost.

<img src='../../img/xgboost_meme.jpg' width=40% />

In [1]:
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [2]:
train = pd.read_csv('../../data/flight_delays_train.csv')
test = pd.read_csv('../../data/flight_delays_test.csv')

In [3]:
train.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y


In [4]:
X_train, X_test = train.drop('dep_delayed_15min', axis=1), test.copy()

In [5]:
y_train = train['dep_delayed_15min'].map({'Y': 1, 'N': 0})

In [6]:
from catboost import CatBoostClassifier

In [7]:
ctb = CatBoostClassifier(random_seed=17)

In [8]:
train_df, y = train.drop('dep_delayed_15min', axis=1), train['dep_delayed_15min'].map({'Y': 1, 'N': 0})
train_df_part, valid_df, y_train_part, y_valid = train_test_split(train_df, y, 
                                                                  test_size=.3, stratify=y, 
                                                                  random_state=17)

In [9]:
cat_features_idx = np.where(train_df_part.dtypes == 'object')[0].tolist()

In [10]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=17)

In [11]:
%%time
cv_scores = []
for train_idx, test_idx in skf.split(train_df_part, y_train_part):
    cv_train_df, cv_valid_df = train_df_part.iloc[train_idx, :], train_df_part.iloc[test_idx, :]
    y_cv_train, y_cv_valid = y_train_part.iloc[train_idx], y_train_part.iloc[test_idx]
    
    ctb.fit(cv_train_df, y_cv_train,
        cat_features=cat_features_idx);
    
    cv_scores.append(roc_auc_score(y_cv_valid, ctb.predict_proba(cv_valid_df)[:, 1]))

Learning rate set to 0.052343
0:	learn: 0.6644024	total: 145ms	remaining: 2m 24s
1:	learn: 0.6388047	total: 231ms	remaining: 1m 55s
2:	learn: 0.6165361	total: 314ms	remaining: 1m 44s
3:	learn: 0.5975645	total: 399ms	remaining: 1m 39s
4:	learn: 0.5808465	total: 476ms	remaining: 1m 34s
5:	learn: 0.5653119	total: 588ms	remaining: 1m 37s
6:	learn: 0.5522280	total: 675ms	remaining: 1m 35s
7:	learn: 0.5408060	total: 759ms	remaining: 1m 34s
8:	learn: 0.5303571	total: 848ms	remaining: 1m 33s
9:	learn: 0.5217073	total: 931ms	remaining: 1m 32s
10:	learn: 0.5134809	total: 1.01s	remaining: 1m 31s
11:	learn: 0.5061019	total: 1.09s	remaining: 1m 29s
12:	learn: 0.4999892	total: 1.18s	remaining: 1m 29s
13:	learn: 0.4942797	total: 1.29s	remaining: 1m 31s
14:	learn: 0.4893480	total: 1.38s	remaining: 1m 30s
15:	learn: 0.4845291	total: 1.46s	remaining: 1m 30s
16:	learn: 0.4805101	total: 1.55s	remaining: 1m 29s
17:	learn: 0.4772721	total: 1.63s	remaining: 1m 28s
18:	learn: 0.4743221	total: 1.74s	remaining:

159:	learn: 0.4219166	total: 15.4s	remaining: 1m 21s
160:	learn: 0.4218238	total: 15.5s	remaining: 1m 20s
161:	learn: 0.4217716	total: 15.6s	remaining: 1m 20s
162:	learn: 0.4216606	total: 15.7s	remaining: 1m 20s
163:	learn: 0.4216058	total: 15.8s	remaining: 1m 20s
164:	learn: 0.4215331	total: 15.9s	remaining: 1m 20s
165:	learn: 0.4214915	total: 16s	remaining: 1m 20s
166:	learn: 0.4214387	total: 16.1s	remaining: 1m 20s
167:	learn: 0.4214013	total: 16.2s	remaining: 1m 20s
168:	learn: 0.4213615	total: 16.3s	remaining: 1m 20s
169:	learn: 0.4212836	total: 16.4s	remaining: 1m 20s
170:	learn: 0.4211901	total: 16.5s	remaining: 1m 19s
171:	learn: 0.4211275	total: 16.6s	remaining: 1m 19s
172:	learn: 0.4210100	total: 16.7s	remaining: 1m 19s
173:	learn: 0.4209474	total: 16.8s	remaining: 1m 19s
174:	learn: 0.4208619	total: 16.9s	remaining: 1m 19s
175:	learn: 0.4207395	total: 17s	remaining: 1m 19s
176:	learn: 0.4206626	total: 17.1s	remaining: 1m 19s
177:	learn: 0.4205633	total: 17.1s	remaining: 1m 1

315:	learn: 0.4136562	total: 30.7s	remaining: 1m 6s
316:	learn: 0.4136311	total: 30.8s	remaining: 1m 6s
317:	learn: 0.4136078	total: 30.9s	remaining: 1m 6s
318:	learn: 0.4135652	total: 31s	remaining: 1m 6s
319:	learn: 0.4135235	total: 31.1s	remaining: 1m 6s
320:	learn: 0.4134840	total: 31.2s	remaining: 1m 5s
321:	learn: 0.4134112	total: 31.3s	remaining: 1m 5s
322:	learn: 0.4133862	total: 31.3s	remaining: 1m 5s
323:	learn: 0.4133141	total: 31.4s	remaining: 1m 5s
324:	learn: 0.4132686	total: 31.6s	remaining: 1m 5s
325:	learn: 0.4132356	total: 31.6s	remaining: 1m 5s
326:	learn: 0.4131483	total: 31.7s	remaining: 1m 5s
327:	learn: 0.4131128	total: 31.8s	remaining: 1m 5s
328:	learn: 0.4130512	total: 31.9s	remaining: 1m 5s
329:	learn: 0.4130026	total: 32s	remaining: 1m 5s
330:	learn: 0.4129919	total: 32.1s	remaining: 1m 4s
331:	learn: 0.4129245	total: 32.2s	remaining: 1m 4s
332:	learn: 0.4128908	total: 32.3s	remaining: 1m 4s
333:	learn: 0.4128642	total: 32.4s	remaining: 1m 4s
334:	learn: 0.41

475:	learn: 0.4069974	total: 46.3s	remaining: 51s
476:	learn: 0.4069522	total: 46.4s	remaining: 50.9s
477:	learn: 0.4069251	total: 46.5s	remaining: 50.8s
478:	learn: 0.4069051	total: 46.6s	remaining: 50.7s
479:	learn: 0.4068584	total: 46.7s	remaining: 50.6s
480:	learn: 0.4068350	total: 46.8s	remaining: 50.5s
481:	learn: 0.4068094	total: 46.9s	remaining: 50.4s
482:	learn: 0.4067691	total: 47s	remaining: 50.3s
483:	learn: 0.4067103	total: 47.1s	remaining: 50.2s
484:	learn: 0.4066672	total: 47.2s	remaining: 50.1s
485:	learn: 0.4066429	total: 47.3s	remaining: 50s
486:	learn: 0.4066094	total: 47.4s	remaining: 50s
487:	learn: 0.4065530	total: 47.5s	remaining: 49.8s
488:	learn: 0.4065149	total: 47.6s	remaining: 49.8s
489:	learn: 0.4064601	total: 47.8s	remaining: 49.7s
490:	learn: 0.4064249	total: 47.9s	remaining: 49.7s
491:	learn: 0.4063940	total: 48.1s	remaining: 49.6s
492:	learn: 0.4063609	total: 48.1s	remaining: 49.5s
493:	learn: 0.4062999	total: 48.3s	remaining: 49.4s
494:	learn: 0.406286

637:	learn: 0.4010199	total: 1m 3s	remaining: 35.8s
638:	learn: 0.4009608	total: 1m 3s	remaining: 35.7s
639:	learn: 0.4009268	total: 1m 3s	remaining: 35.6s
640:	learn: 0.4009010	total: 1m 3s	remaining: 35.5s
641:	learn: 0.4008745	total: 1m 3s	remaining: 35.4s
642:	learn: 0.4008507	total: 1m 3s	remaining: 35.3s
643:	learn: 0.4008326	total: 1m 3s	remaining: 35.2s
644:	learn: 0.4008203	total: 1m 3s	remaining: 35.1s
645:	learn: 0.4007779	total: 1m 3s	remaining: 35s
646:	learn: 0.4007552	total: 1m 3s	remaining: 34.9s
647:	learn: 0.4007068	total: 1m 4s	remaining: 34.8s
648:	learn: 0.4006581	total: 1m 4s	remaining: 34.7s
649:	learn: 0.4006078	total: 1m 4s	remaining: 34.6s
650:	learn: 0.4005646	total: 1m 4s	remaining: 34.5s
651:	learn: 0.4005374	total: 1m 4s	remaining: 34.4s
652:	learn: 0.4005062	total: 1m 4s	remaining: 34.3s
653:	learn: 0.4004866	total: 1m 4s	remaining: 34.2s
654:	learn: 0.4004571	total: 1m 4s	remaining: 34.1s
655:	learn: 0.4004522	total: 1m 4s	remaining: 34s
656:	learn: 0.40

...

266:	learn: 0.4147771	total: 24.3s	remaining: 1m 6s
267:	learn: 0.4147308	total: 24.4s	remaining: 1m 6s
268:	learn: 0.4146685	total: 24.5s	remaining: 1m 6s
269:	learn: 0.4146207	total: 24.6s	remaining: 1m 6s
270:	learn: 0.4145740	total: 24.7s	remaining: 1m 6s
271:	learn: 0.4145442	total: 24.8s	remaining: 1m 6s
272:	learn: 0.4144796	total: 24.9s	remaining: 1m 6s
273:	learn: 0.4144316	total: 24.9s	remaining: 1m 6s
274:	learn: 0.4143478	total: 25s	remaining: 1m 5s
275:	learn: 0.4143054	total: 25.1s	remaining: 1m 5s
276:	learn: 0.4142622	total: 25.2s	remaining: 1m 5s
277:	learn: 0.4141991	total: 25.3s	remaining: 1m 5s
278:	learn: 0.4141573	total: 25.4s	remaining: 1m 5s
279:	learn: 0.4141243	total: 25.4s	remaining: 1m 5s
280:	learn: 0.4140701	total: 25.5s	remaining: 1m 5s
281:	learn: 0.4140175	total: 25.6s	remaining: 1m 5s
282:	learn: 0.4139537	total: 25.7s	remaining: 1m 5s
283:	learn: 0.4139259	total: 25.8s	remaining: 1m 5s
284:	learn: 0.4138917	total: 25.9s	remaining: 1m 4s
285:	learn: 0.

425:	learn: 0.4077893	total: 39.3s	remaining: 52.9s
426:	learn: 0.4077624	total: 39.4s	remaining: 52.8s
427:	learn: 0.4077153	total: 39.5s	remaining: 52.7s
428:	learn: 0.4076913	total: 39.6s	remaining: 52.7s
429:	learn: 0.4076529	total: 39.7s	remaining: 52.6s
430:	learn: 0.4076313	total: 39.8s	remaining: 52.5s
431:	learn: 0.4075977	total: 39.9s	remaining: 52.4s
432:	learn: 0.4075049	total: 40s	remaining: 52.3s
433:	learn: 0.4074937	total: 40.1s	remaining: 52.2s
434:	learn: 0.4074598	total: 40.1s	remaining: 52.1s
435:	learn: 0.4074212	total: 40.3s	remaining: 52.1s
436:	learn: 0.4073850	total: 40.4s	remaining: 52s
437:	learn: 0.4073237	total: 40.5s	remaining: 51.9s
438:	learn: 0.4072846	total: 40.6s	remaining: 51.8s
439:	learn: 0.4072566	total: 40.7s	remaining: 51.8s
440:	learn: 0.4072257	total: 40.8s	remaining: 51.7s
441:	learn: 0.4071471	total: 40.9s	remaining: 51.6s
442:	learn: 0.4071104	total: 41s	remaining: 51.5s
443:	learn: 0.4070526	total: 41.1s	remaining: 51.4s
444:	learn: 0.4070

585:	learn: 0.4018092	total: 55.4s	remaining: 39.1s
586:	learn: 0.4017724	total: 55.5s	remaining: 39s
587:	learn: 0.4017327	total: 55.6s	remaining: 38.9s
588:	learn: 0.4016863	total: 55.7s	remaining: 38.8s
589:	learn: 0.4016306	total: 55.8s	remaining: 38.8s
590:	learn: 0.4016018	total: 55.9s	remaining: 38.7s
591:	learn: 0.4015685	total: 56s	remaining: 38.6s
592:	learn: 0.4015147	total: 56.1s	remaining: 38.5s
593:	learn: 0.4014845	total: 56.2s	remaining: 38.4s
594:	learn: 0.4014547	total: 56.3s	remaining: 38.3s
595:	learn: 0.4014261	total: 56.4s	remaining: 38.2s
596:	learn: 0.4014160	total: 56.5s	remaining: 38.1s
597:	learn: 0.4013670	total: 56.6s	remaining: 38s
598:	learn: 0.4013215	total: 56.7s	remaining: 37.9s
599:	learn: 0.4012897	total: 56.8s	remaining: 37.9s
600:	learn: 0.4012669	total: 56.9s	remaining: 37.8s
601:	learn: 0.4012267	total: 57s	remaining: 37.7s
602:	learn: 0.4011967	total: 57.1s	remaining: 37.6s
603:	learn: 0.4011682	total: 57.2s	remaining: 37.5s
604:	learn: 0.401139

744:	learn: 0.3958941	total: 1m 10s	remaining: 24.1s
745:	learn: 0.3958828	total: 1m 10s	remaining: 24s
746:	learn: 0.3958445	total: 1m 10s	remaining: 23.9s
747:	learn: 0.3957953	total: 1m 10s	remaining: 23.8s
748:	learn: 0.3957446	total: 1m 10s	remaining: 23.7s
749:	learn: 0.3957117	total: 1m 10s	remaining: 23.6s
750:	learn: 0.3957047	total: 1m 10s	remaining: 23.5s
751:	learn: 0.3956648	total: 1m 10s	remaining: 23.4s
752:	learn: 0.3956120	total: 1m 11s	remaining: 23.3s
753:	learn: 0.3955671	total: 1m 11s	remaining: 23.2s
754:	learn: 0.3955365	total: 1m 11s	remaining: 23.1s
755:	learn: 0.3954920	total: 1m 11s	remaining: 23s
756:	learn: 0.3954442	total: 1m 11s	remaining: 22.9s
757:	learn: 0.3954127	total: 1m 11s	remaining: 22.8s
758:	learn: 0.3953816	total: 1m 11s	remaining: 22.7s
759:	learn: 0.3953674	total: 1m 11s	remaining: 22.6s
760:	learn: 0.3953111	total: 1m 11s	remaining: 22.5s
761:	learn: 0.3952835	total: 1m 11s	remaining: 22.4s
762:	learn: 0.3952371	total: 1m 11s	remaining: 22.

900:	learn: 0.3906843	total: 1m 25s	remaining: 9.34s
901:	learn: 0.3906540	total: 1m 25s	remaining: 9.25s
902:	learn: 0.3906223	total: 1m 25s	remaining: 9.16s
903:	learn: 0.3905935	total: 1m 25s	remaining: 9.06s
904:	learn: 0.3905586	total: 1m 25s	remaining: 8.97s
905:	learn: 0.3905380	total: 1m 25s	remaining: 8.87s
906:	learn: 0.3905122	total: 1m 25s	remaining: 8.78s
907:	learn: 0.3904818	total: 1m 25s	remaining: 8.69s
908:	learn: 0.3904333	total: 1m 25s	remaining: 8.59s
909:	learn: 0.3904032	total: 1m 25s	remaining: 8.5s
910:	learn: 0.3903868	total: 1m 25s	remaining: 8.4s
911:	learn: 0.3903498	total: 1m 26s	remaining: 8.31s
912:	learn: 0.3902986	total: 1m 26s	remaining: 8.21s
913:	learn: 0.3902674	total: 1m 26s	remaining: 8.12s
914:	learn: 0.3902174	total: 1m 26s	remaining: 8.03s
915:	learn: 0.3901531	total: 1m 26s	remaining: 7.93s
916:	learn: 0.3901281	total: 1m 26s	remaining: 7.84s
917:	learn: 0.3900903	total: 1m 26s	remaining: 7.74s
918:	learn: 0.3900601	total: 1m 26s	remaining: 7

In [12]:
np.mean(cv_scores)

0.7540041873779526

In [13]:
%%time
ctb.fit(train_df_part, y_train_part,
        cat_features=cat_features_idx);

Learning rate set to 0.055755
0:	learn: 0.6626904	total: 118ms	remaining: 1m 58s
1:	learn: 0.6357374	total: 214ms	remaining: 1m 46s
2:	learn: 0.6126228	total: 320ms	remaining: 1m 46s
3:	learn: 0.5920469	total: 411ms	remaining: 1m 42s
4:	learn: 0.5748898	total: 509ms	remaining: 1m 41s
5:	learn: 0.5596350	total: 622ms	remaining: 1m 42s
6:	learn: 0.5462894	total: 733ms	remaining: 1m 44s
7:	learn: 0.5347138	total: 840ms	remaining: 1m 44s
8:	learn: 0.5243772	total: 950ms	remaining: 1m 44s
9:	learn: 0.5155100	total: 1.07s	remaining: 1m 45s
10:	learn: 0.5073349	total: 1.17s	remaining: 1m 45s
11:	learn: 0.5008048	total: 1.27s	remaining: 1m 44s
12:	learn: 0.4948434	total: 1.38s	remaining: 1m 44s
13:	learn: 0.4892045	total: 1.48s	remaining: 1m 43s
14:	learn: 0.4843305	total: 1.58s	remaining: 1m 43s
15:	learn: 0.4800113	total: 1.67s	remaining: 1m 42s
16:	learn: 0.4758255	total: 1.76s	remaining: 1m 41s
17:	learn: 0.4728804	total: 1.86s	remaining: 1m 41s
18:	learn: 0.4696907	total: 1.95s	remaining:

157:	learn: 0.4203853	total: 17.7s	remaining: 1m 34s
158:	learn: 0.4203422	total: 17.8s	remaining: 1m 34s
159:	learn: 0.4202861	total: 17.9s	remaining: 1m 34s
160:	learn: 0.4202371	total: 18s	remaining: 1m 33s
161:	learn: 0.4201815	total: 18.1s	remaining: 1m 33s
162:	learn: 0.4201049	total: 18.3s	remaining: 1m 33s
163:	learn: 0.4200192	total: 18.4s	remaining: 1m 33s
164:	learn: 0.4199332	total: 18.5s	remaining: 1m 33s
165:	learn: 0.4198797	total: 18.6s	remaining: 1m 33s
166:	learn: 0.4198428	total: 18.7s	remaining: 1m 33s
167:	learn: 0.4197510	total: 18.8s	remaining: 1m 33s
168:	learn: 0.4197015	total: 18.9s	remaining: 1m 33s
169:	learn: 0.4195833	total: 19s	remaining: 1m 32s
170:	learn: 0.4195432	total: 19.1s	remaining: 1m 32s
171:	learn: 0.4195196	total: 19.2s	remaining: 1m 32s
172:	learn: 0.4194506	total: 19.3s	remaining: 1m 32s
173:	learn: 0.4194298	total: 19.4s	remaining: 1m 32s
174:	learn: 0.4193615	total: 19.6s	remaining: 1m 32s
175:	learn: 0.4193268	total: 19.7s	remaining: 1m 3

313:	learn: 0.4121388	total: 35.7s	remaining: 1m 18s
314:	learn: 0.4120896	total: 35.8s	remaining: 1m 17s
315:	learn: 0.4120568	total: 36s	remaining: 1m 17s
316:	learn: 0.4119827	total: 36.1s	remaining: 1m 17s
317:	learn: 0.4119586	total: 36.2s	remaining: 1m 17s
318:	learn: 0.4119366	total: 36.3s	remaining: 1m 17s
319:	learn: 0.4118904	total: 36.4s	remaining: 1m 17s
320:	learn: 0.4118400	total: 36.5s	remaining: 1m 17s
321:	learn: 0.4118230	total: 36.6s	remaining: 1m 17s
322:	learn: 0.4117769	total: 36.7s	remaining: 1m 16s
323:	learn: 0.4117212	total: 36.8s	remaining: 1m 16s
324:	learn: 0.4116935	total: 36.9s	remaining: 1m 16s
325:	learn: 0.4116468	total: 37s	remaining: 1m 16s
326:	learn: 0.4115906	total: 37.1s	remaining: 1m 16s
327:	learn: 0.4115502	total: 37.2s	remaining: 1m 16s
328:	learn: 0.4115039	total: 37.4s	remaining: 1m 16s
329:	learn: 0.4114610	total: 37.5s	remaining: 1m 16s
330:	learn: 0.4114350	total: 37.6s	remaining: 1m 15s
331:	learn: 0.4113689	total: 37.7s	remaining: 1m 1

472:	learn: 0.4063453	total: 53.7s	remaining: 59.8s
473:	learn: 0.4062853	total: 53.8s	remaining: 59.7s
474:	learn: 0.4062619	total: 54s	remaining: 59.6s
475:	learn: 0.4062349	total: 54.1s	remaining: 59.5s
476:	learn: 0.4061925	total: 54.2s	remaining: 59.4s
477:	learn: 0.4061516	total: 54.3s	remaining: 59.3s
478:	learn: 0.4060872	total: 54.5s	remaining: 59.2s
479:	learn: 0.4060527	total: 54.6s	remaining: 59.1s
480:	learn: 0.4060253	total: 54.7s	remaining: 59s
481:	learn: 0.4059734	total: 54.8s	remaining: 58.9s
482:	learn: 0.4059595	total: 54.9s	remaining: 58.8s
483:	learn: 0.4059001	total: 55s	remaining: 58.7s
484:	learn: 0.4058595	total: 55.1s	remaining: 58.5s
485:	learn: 0.4058444	total: 55.2s	remaining: 58.4s
486:	learn: 0.4058155	total: 55.4s	remaining: 58.3s
487:	learn: 0.4057751	total: 55.5s	remaining: 58.2s
488:	learn: 0.4057478	total: 55.6s	remaining: 58.1s
489:	learn: 0.4056889	total: 55.8s	remaining: 58s
490:	learn: 0.4056610	total: 55.9s	remaining: 57.9s
491:	learn: 0.405623

632:	learn: 0.4012432	total: 1m 12s	remaining: 42s
633:	learn: 0.4011966	total: 1m 12s	remaining: 41.8s
634:	learn: 0.4011842	total: 1m 12s	remaining: 41.7s
635:	learn: 0.4011512	total: 1m 12s	remaining: 41.6s
636:	learn: 0.4011301	total: 1m 12s	remaining: 41.5s
637:	learn: 0.4010920	total: 1m 12s	remaining: 41.4s
638:	learn: 0.4010511	total: 1m 13s	remaining: 41.3s
639:	learn: 0.4010335	total: 1m 13s	remaining: 41.2s
640:	learn: 0.4010099	total: 1m 13s	remaining: 41s
641:	learn: 0.4009768	total: 1m 13s	remaining: 40.9s
642:	learn: 0.4009342	total: 1m 13s	remaining: 40.8s
643:	learn: 0.4009172	total: 1m 13s	remaining: 40.7s
644:	learn: 0.4008612	total: 1m 13s	remaining: 40.6s
645:	learn: 0.4008371	total: 1m 13s	remaining: 40.5s
646:	learn: 0.4008122	total: 1m 13s	remaining: 40.3s
647:	learn: 0.4007979	total: 1m 14s	remaining: 40.2s
648:	learn: 0.4007703	total: 1m 14s	remaining: 40.1s
649:	learn: 0.4007202	total: 1m 14s	remaining: 40s
650:	learn: 0.4006873	total: 1m 14s	remaining: 39.9s

789:	learn: 0.3959703	total: 1m 30s	remaining: 24.1s
790:	learn: 0.3959351	total: 1m 30s	remaining: 24s
791:	learn: 0.3959224	total: 1m 31s	remaining: 23.9s
792:	learn: 0.3958848	total: 1m 31s	remaining: 23.8s
793:	learn: 0.3958591	total: 1m 31s	remaining: 23.7s
794:	learn: 0.3958189	total: 1m 31s	remaining: 23.6s
795:	learn: 0.3957690	total: 1m 31s	remaining: 23.5s
796:	learn: 0.3957442	total: 1m 31s	remaining: 23.3s
797:	learn: 0.3957286	total: 1m 31s	remaining: 23.2s
798:	learn: 0.3957001	total: 1m 31s	remaining: 23.1s
799:	learn: 0.3956658	total: 1m 32s	remaining: 23s
800:	learn: 0.3956485	total: 1m 32s	remaining: 22.9s
801:	learn: 0.3956239	total: 1m 32s	remaining: 22.8s
802:	learn: 0.3956013	total: 1m 32s	remaining: 22.7s
803:	learn: 0.3955764	total: 1m 32s	remaining: 22.6s
804:	learn: 0.3955231	total: 1m 32s	remaining: 22.4s
805:	learn: 0.3955065	total: 1m 32s	remaining: 22.3s
806:	learn: 0.3954585	total: 1m 32s	remaining: 22.2s
807:	learn: 0.3954262	total: 1m 32s	remaining: 22.

946:	learn: 0.3911595	total: 1m 50s	remaining: 6.19s
947:	learn: 0.3911475	total: 1m 50s	remaining: 6.08s
948:	learn: 0.3911265	total: 1m 50s	remaining: 5.96s
949:	learn: 0.3911001	total: 1m 51s	remaining: 5.84s
950:	learn: 0.3910771	total: 1m 51s	remaining: 5.73s
951:	learn: 0.3910286	total: 1m 51s	remaining: 5.61s
952:	learn: 0.3910085	total: 1m 51s	remaining: 5.49s
953:	learn: 0.3909534	total: 1m 51s	remaining: 5.38s
954:	learn: 0.3909309	total: 1m 51s	remaining: 5.26s
955:	learn: 0.3909108	total: 1m 51s	remaining: 5.14s
956:	learn: 0.3908908	total: 1m 51s	remaining: 5.03s
957:	learn: 0.3908618	total: 1m 51s	remaining: 4.91s
958:	learn: 0.3908297	total: 1m 52s	remaining: 4.79s
959:	learn: 0.3908020	total: 1m 52s	remaining: 4.67s
960:	learn: 0.3907829	total: 1m 52s	remaining: 4.56s
961:	learn: 0.3907390	total: 1m 52s	remaining: 4.44s
962:	learn: 0.3907177	total: 1m 52s	remaining: 4.32s
963:	learn: 0.3906857	total: 1m 52s	remaining: 4.21s
964:	learn: 0.3906494	total: 1m 52s	remaining:

In [14]:
roc_auc_score(y_valid, ctb.predict_proba(valid_df)[:, 1])

0.7657513806089964

In [15]:
test_pred = ctb.predict_proba(X_test)[:, 1]

In [16]:
pd.Series(test_pred, 
          name='dep_delayed_15min').to_csv('catboost.csv', 
                                           index_label='id', 
                                           header=True)

0.73036